In [51]:
# %%
import osmnx as ox
import pandas as pd
from routing import astar_route
import folium
from shapely.geometry import LineString
import matplotlib.pyplot as plt

# 1. Load the LA driving graph, takes 2-4 minutes!!!
G = ox.graph_from_place("Los Angeles, California, USA", network_type="drive")

In [52]:
print(G)

MultiDiGraph with 49474 nodes and 135861 edges


In [ ]:

def get_edge_data(G):

    # ## 2. Load the risk map for a given weather condition
    condition = "Overcast"  # ← pick one of: Clear, Partially cloudy, Overcast, Rain, Rain, Overcast, Rain, Partially cloudy
    csv_path = f"risk_maps/{condition.replace(', ', '_').replace(' ', '_')}.csv"
    risk_df = pd.read_csv(csv_path)

    # Build a lookup from road_name → risk_score
    risk_lookup = dict(zip(risk_df["road_name"], risk_df["risk_score"]))

    # ## 3. Annotate every edge with two cost fields
    for u, v, key, data in G.edges(keys=True, data=True):
        # base length
        length = data.get("length", 1.0)
        data["cost_distance"] = length

        # determine road name
        name = data.get("name")
        if isinstance(name, list):
            name = name[0]
        if not name:
            name = "Unnamed Road"

        # lookup risk (default = 0)
        risk = risk_lookup.get(name, 0.0)

        # cost_risk = length * (1 + risk) so that risk 0→1 scales cost 1×→2×
        data["cost_risk"] = length * (1 + risk)


    # --- Add travel time cost ---
        # Get maxspeed (can be a list or string, in km/h)
        maxspeed = data.get("maxspeed")
        if isinstance(maxspeed, list):
            try:
                maxspeed = float(maxspeed[0])
            except:
                maxspeed = None
        try:
            maxspeed = float(maxspeed)
        except:
            maxspeed = None
        if not maxspeed or maxspeed <= 0:
            maxspeed = 50  # default to 50 km/h if missing

        # Calculate time in hours: time = distance (km) / speed (km/h)
        time_hours = (length / 1000) / maxspeed
        data["cost_time"] = time_hours
    return G

# %% [markdown]
# ## 4. Pick your cost attribute based on the condition
# if you want pure distance:
#   cost_attr = "cost_distance"
#
# if you want to penalize risky roads:
#   cost_attr = "cost_risk"


def path_finding(G, origin, dest, use_risk):
    G = get_edge_data(G)
    cost_attr = "cost_risk" if use_risk else "cost_distance"

    #origin = (34.06655181863482, -118.18789017013762)
    #dest = (34.0647417476385, -118.25823027030046)

    if use_risk:
        node_path, edge_path, road_names = astar_route(
            G,
            origin_point=origin,
            destination_point=dest,
            weight=cost_attr
        )
        print(node_path)
        print(f"Number of nodes in path: {len(node_path)}")
        print(f"Number of edges in path: {len(edge_path)}")
        print("Roads to follow:", road_names)

    # Compute fastest route (minimize travel time)
    node_path_fast, edge_path_fast, road_names_fast = astar_route(
        G,
        origin_point=origin,
        destination_point=dest,
        weight="cost_length",
    )
    
    print(node_path_fast)
    print(f"Number of nodes in path (fastest route): {len(node_path_fast)}")
    print(f"Number of edges in path (fastest route): {len(node_path_fast)}")
    print("Roads to follow(fastest route): ", road_names_fast)

    return G, edge_path, edge_path_fast

In [60]:
# Define a color scale for risk (0.0 = green, 1.0+ = red)
def risk_to_color(risk, origin):
    # Clamp risk between 0 and 1
    risk = min(max(risk, 0), 1)
    # Use a color map (green to red)
    cmap = plt.get_cmap("RdYlGn_r")  # reverse of green→red
    r, g, b, _ = cmap(risk)
    return f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}'


def VisuzalizeMap(G, origin, dest, use_risk=True):
    G, edge_path, edge_path_fast = path_finding(G, origin, dest, use_risk=use_risk)

    # Calculate total travel time for each route (in minutes)
    total_time_risk = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path) * 60
    total_time_fast = sum(G.edges[u, v, key]["cost_time"] for u, v, key in edge_path_fast) * 60

    # Create a folium map centered on the origin
    m = folium.Map(location=origin, zoom_start=13)

    ## Risk Route Coloring ###
    for u, v, key in edge_path:
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            line = LineString([point_u, point_v])
        length = data.get("length", 1.0)
        cost_risk = data.get("cost_risk", length)
        base_risk = (cost_risk / length) - 1
        color = risk_to_color(base_risk, origin)
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in line.coords],
            color=color,
            weight=5,
            opacity=0.8,
            popup=f"{data.get('name', 'Unnamed Road')}<br>Risk: {base_risk:.2f}"
        ).add_to(m)

    ### Fastest Route Coloring ###
    for u, v, key in edge_path_fast:
        data = G.edges[u, v, key]
        if 'geometry' in data:
            line = data['geometry']
        else:
            point_u = (G.nodes[u]['y'], G.nodes[u]['x'])
            point_v = (G.nodes[v]['y'], G.nodes[v]['x'])
            line = LineString([point_u, point_v])
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in line.coords],
            color="blue",
            weight=3,
            opacity=0.7,
            popup=f"Fastest: {data.get('name', 'Unnamed Road')}"
        ).add_to(m)

    # Add start and end markers
    folium.Marker(origin, tooltip="Start", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(dest, tooltip="Destination", icon=folium.Icon(color="red")).add_to(m)

    # Add info box with travel times
    info_html = f"""
    <div style="position: fixed; 
                top: 30px; left: 30px; width: 250px; height: 80px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Route Time Estimates</b><br>
    <span style='color:black;'>Safest route: </span>{total_time_risk:.1f} min<br>
    <span style='color:blue;'>Fastest route: </span>{total_time_fast:.1f} min
    </div>
    """
    m.get_root().html.add_child(folium.Element(info_html))

    # Risk legend
    legend_html = """
    <div style="position: fixed; 
                bottom: 30px; left: 30px; width: 150px; height: 100px; 
                background-color: white; z-index:9999; font-size:14px;
                border:2px solid grey; padding: 10px;">
    <b>Risk Level</b><br>
    <span style="color:#00ff00;">●</span> Low<br>
    <span style="color:#ffff00;">●</span> Medium<br>
    <span style="color:#ff0000;">●</span> High
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    # Map styles
    folium.TileLayer("Stamen Terrain").add_to(m)
    folium.TileLayer("CartoDB Positron").add_to(m)
    folium.TileLayer("openstreetmap").add_to(m)
    folium.LayerControl().add_to(m)

    # Save to HTML
    m.save("risky_path.html")
    print("Map saved to risky_path.html")


In [61]:
origin = (34.06655181863482, -118.18789017013762)
dest = (34.0647417476385, -118.25823027030046)

VisuzalizeMap(G, origin, dest, use_risk=True)

[1849409643, 123018936, 123565234, 33155647, 14941504, 1985260166, 2296327020, 1985260196, 281506189, 2258860847, 281506167, 32274961, 72448807, 14798642, 72448746, 17737741, 14941693, 21150896, 16941224, 122977613, 16203358, 122741782, 122977609, 68965696, 1994531073, 250001395, 122656711, 122872202, 122977592, 122977589, 122967648, 122792131, 122977587, 122670717, 268373135, 17677673, 11340695338, 68267399, 122645236, 123637708, 68267394, 1838396767, 60945762, 268653344, 250586509, 268653330, 1842838208, 1838389220, 20400044, 20400214, 60946365, 122663619, 60946368, 122999186, 123008568, 123008570, 122834000, 123008579, 123008582]
Number of nodes in path: 59
Number of edges in path: 58
Roads to follow: ['North Rowan Avenue', 'North Rowan Avenue', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'Valley Boulevard', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Street', 'North Main Str